In [2]:
import requests
from pathlib import Path
import json
import geopandas as gpd
import pandas as pd


basePath = Path.cwd().parent.parent.parent / 'inputs/osm_rails'
basePath.mkdir(parents=True, exist_ok=True)

In [ ]:
URL = 'https://wwf-sight-maps.org/arcgis/rest/services/Global/Global_Railways/MapServer/0/query'


offset = 0
hasMore = True
while hasMore:
  resp = requests.get(URL, params={
    'outFields':'OBJECTID',
    'where':'1=1',
    'f':'geojson',
    'resultOffset': offset
  }).json()
  with open(basePath / f'{int(offset / 1000)}.json', 'w') as f:
    json.dump(resp, f)
  offset += len(resp['features'])
  hasMore = 'exceededTransferLimit' in resp
  print("Downloaded file no.", int(offset / 1000))

In [4]:
giantGpd = pd.concat([gpd.read_file(basePath / f'{f}.json') for f in range(3113)])

In [6]:
osmRaildf = pd.read_csv(basePath.parent / 'osm_rails.csv')

/tmp/ipykernel_574395/3390000887.py:1: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  osmRaildf = pd.read_csv(basePath.parent / 'osm_rails.csv')


In [9]:
osmRaildf

,OBJECTID,Railway,Electrified,Shape_Length,Length_Km_Railways,Gauge
0,1,rail,no,760.719371,0.733608,1067
1,2,light_rail,contact_line,13.845271,0.013352,1435
2,3,subway,contact_line,149.546862,0.138368,1432
3,4,subway,contact_line,1029.851794,0.952705,1432
4,5,subway,contact_line,1018.054601,0.939677,1432
...,...,...,...,...,...,...
3113952,3113953,NaN,NaN,39.679955,0.036549,NaN
3113953,3113954,NaN,NaN,10.349299,0.009532,NaN
3113954,3113955,NaN,NaN,842349.092126,809.023008,NaN
3113955,3113956,NaN,NaN,6722.031031,5.909905,NaN


In [10]:
giantGpd = giantGpd.set_index('OBJECTID').join(osmRaildf.set_index('OBJECTID'))
giantGpd.head()

,geometry,Railway,Electrified,Shape_Length,Length_Km_Railways,Gauge
OBJECTID,,,,,,
1,"LINESTRING (121.00925 14.59088, 121.00892 14.5...",rail,no,760.719371,0.733608,1067
2,"LINESTRING (120.99863 14.53466, 120.99871 14.5...",light_rail,contact_line,13.845271,0.013352,1435
3,"LINESTRING (113.94352 22.3223, 113.94221 22.32...",subway,contact_line,149.546862,0.138368,1432
4,"LINESTRING (114.05075 22.34109, 114.05044 22.3...",subway,contact_line,1029.851794,0.952705,1432
5,"LINESTRING (114.12591 22.34832, 114.12518 22.3...",subway,contact_line,1018.054601,0.939677,1432


In [3]:
shpFilePath = basePath.parent / 'global_osm_rails'
shpFilePath.mkdir(parents=True, exist_ok=True)

In [ ]:
giantGpd.to_file(shpFilePath / 'global_osm_rails.shp')

In [4]:
osmRailDf = gpd.read_file(shpFilePath / 'global_osm_rails.shp')

In [1]:
len(osmRailDf)

NameError: name 'osmRailDf' is not defined

In [ ]:
frenchRailDf = osmRailDf.cx[1.327149:3.269531,48.461703:49.244361]
print(len(frenchRailDf))
frenchRailDf.sample(5000).explore()